<h2>C - Network

In [18]:
import pandas as pd
import icecream as ic
import numpy as np
import plotly
import matplotlib as plt
import xml.etree.ElementTree as ET
import pandas as pd
import psycopg2
import psycopg2.extras

C1: small-molecule roles

In [61]:
#chebi database

hostname = 'localhost'
database = 'Chebi'
username = 'postgres'
pwd = 'wlf1115'
port_id = 5432
conn = None

query = 'Acetazolamide'
with psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id) as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as cur:

        query_script = '''SELECT type, final_id
                        FROM relation
                        WHERE init_id = (
                            SELECT compound_id
                            FROM names
                            WHERE name IN (%s)
                            )'''
        cur.execute(query_script, (query,))
        
        type_id = {}
        type_compound = {}
        for record in cur.fetchall():
            type_id[record[0]] = []
            type_compound[record[0]] = []
            type_id[record[0]] = record[1]


        for k,v in type_id.items():
            # query1 = '''SELECT name
            #             FROM names
            #             WHERE compound_id IN ({}
            #             )'''.format(tuple([x]))
            query_script1 = '''SELECT name
                        FROM names
                        WHERE compound_id IN (%s)
                        '''
            query1 = tuple([v])
            cur.execute(query_script1, query1)
            for record in cur.fetchall():
                type_compound[k].append(record)

print(query)
type_compound    

Acetazolamide


{'has_part': [['Diamox'],
  ['Sodium acetazolamide'],
  ['sodium [(5-acetamido-1,3,4-thiadiazol-2-yl)sulfonyl]azanide'],
  ['N-(5-Sulfamoyl-1,3,4-thiadiazol-2-yl)acetamide monosodium salt']],
 'is_conjugate_base_of': [['[(5-acetamido-1,3,4-thiadiazol-2-yl)sulfonyl]azanide'],
  ['acetazolamide']]}

C2: metabolic pathways

In [ ]:
# https://www.ebi.ac.uk/biomodels/MODEL1109130000#Files #!how to get binary data, how is the graph represented # read paper
# We downloaded the reconstruction of human metabolism (Recon)57 from Pathway Commons58 (http://www.pathwaycommons.org, July 2017) in binary interaction form. 
# Data were represented as an undirected graph where nodes are metabolites and edges denote reactions. 

C3: signaling pathways

In [ ]:
#  C3 space (and the C4 and C5 spaces) is aimed at any molecule with known protein targets. 
# In this case, we list the biological pathways that may be affected by the interaction of a molecule with its targets. 
# Human pathways were collected from Reactome (https://reactome.org, May 2017) #! use of graphDB
# we chose to use binding activities from B4, since this is an extensive dataset containing mostly literature data with well-accepted activity thresholds. 
# In B4, 24.5% of the compound–protein interactions do not correspond to human proteins. #!need B4 data
# These were mapped to their human orthologs using MetaPhOrs60 (http://orthology.phylomedb.org, May 2017), following the observation that binding activities can be safely transferred between orthologous proteins61,  #!how is it done, it is a database?
# especially if they belong to closely related species, as it is the case for B4 data62. Of all the nonhuman proteins mapped to the human orthologs, 94.4% were mammal proteins #!http://orthology.phylomedb.org/download

C4: biological processes

In [ ]:
# We downloaded the Gene Ontology Annotation database (https://www.ebi.ac.uk/GOA, May 2017)  #!data can't be downloaded
# read the ‘biological process’ branch of the ontology as a directed acyclic graph (‘is a’ relationships). 
# Proteins were annotated with their Gene Ontology Annotation biological process terms plus parent terms (up to the root of the directed acyclic graph). 
# Similar to C3, we associated molecules with biological process terms by simply checking the annotations of the molecule targets (B4).

C5: interactomes

In [67]:
# We collected five representative PPI networks, namely STRING (score of >700, that is, high confidence)63 (v.10, https://string-db.org) (14,725 proteins (p), 300,686 interactions (i)), #! waiting for data
# InWeb (score of 0.5)64 (http://www.intomics.com/inbio/map, March 2017) (10,100 p, 168,970 i), #!paid service?
# a portion of Pathway Commons containing interactions from known pathways (Kyoto Encylopedia of Genes and Genomes65, NetPath66, PANTHER67 and WikiPathways68) (9,344 p, 242,962 i), #!need to find out
# an in-house network of physical binary PPIs69 (13,038 p, 64,659 i),  #!need to find in house pipeline
# a network of metabolic genes based on Recon (v.2, http://vmh.uni.lu) (1,628 p, 246,937 i). #! recon https://www.vmh.life/#home matlab file?
import scipy.io
Recon3DModel_mat = scipy.io.loadmat('../Recon3D_301/Recon3DModel_301.mat')
Recon3D_mat = scipy.io.loadmat('../Recon3D_301/Recon3D_301.mat')



# To build this last network, we linked two metabolic proteins (enzymes or transporters) when the product metabolite of the first was the substrate of the second, or when both were needed to perform a certain reaction, suggesting that they are part of the same protein complex. Edges between proteins were weighted inversely proportional to the number of reactions involving their shared metabolites, so that ‘currency’ metabolites such as ATP and water had marginal impact on the network connectivity. To control for indirect associations, we deconvoluted the network using edge weights and setting a network deconvolution score cutoff of 0.9

In [69]:
Recon3D_mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Mon Feb 19 16:58:15 2018',
 '__version__': '1.0',
 '__globals__': [],
 'Recon3D': array([[(<8399x13543 sparse matrix of type '<class 'numpy.float64'>'
         	with 56791 stored elements in Compressed Sparse Column format>, array([[array(['10fthf5glu[c]'], dtype='<U13')],
                [array(['10fthf5glu[l]'], dtype='<U13')],
                [array(['10fthf5glu[m]'], dtype='<U13')],
                ...,
                [array(['protein[c]'], dtype='<U10')],
                [array(['h[i]'], dtype='<U4')],
                [array(['pi[i]'], dtype='<U5')]], dtype=object), array([[0],
                [0],
                [0],
                ...,
                [0],
                [0],
                [0]], dtype=uint8), array(['E', 'E', 'E', ..., 'E', 'E', 'E'], dtype='<U1'), array([[array(['10FTHF5GLUtl'], dtype='<U12')],
                [array(['10FTHF5GLUtm'], dtype='<U12')],
                [array(['10FTHF6GLUtl'

In [68]:
Recon3DModel_mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Mon Feb 19 16:58:46 2018',
 '__version__': '1.0',
 '__globals__': [],
 'Recon3DModel': array([[(<5835x10600 sparse matrix of type '<class 'numpy.float64'>'
         	with 40425 stored elements in Compressed Sparse Column format>, array([[array(['10fthf5glu[c]'], dtype='<U13')],
                [array(['10fthf5glu[l]'], dtype='<U13')],
                [array(['10fthf5glu[m]'], dtype='<U13')],
                ...,
                [array(['1a25dhvitd2[e]'], dtype='<U14')],
                [array(['protein[c]'], dtype='<U10')],
                [array(['h[i]'], dtype='<U4')]], dtype=object), array([[0],
                [0],
                [0],
                ...,
                [0],
                [0],
                [0]], dtype=uint8), array(['E', 'E', 'E', ..., 'E', 'E', 'E'], dtype='<U1'), array([[array(['10FTHF5GLUtl'], dtype='<U12')],
                [array(['10FTHF5GLUtm'], dtype='<U12')],
                [array(

In [63]:
#network processing


SyntaxError: invalid syntax (3283979113.py, line 2)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.2/34.2 MB 32.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
